Preprocessing script for images

First, assign unique names to images using the csv created in the R preprocessing script

In [ ]:
"""
Created on Thu Nov 12, 2020

@author: Sophia Marthine Kleist Karlson
"""
## Script that renames drawings to unique names and puts them all in the same folder


# import the necessary packages
import pandas as pd
import cv2
import os

# set path
path = "C:/Users/Sophia/Documents/Social Transmission Study/Analysis of drawings/" #this script is run locally on my computer, hence this path
os.chdir(path)
print(path)

# create folder for all images to be put in
if not os.path.exists('all_drawings'):
    os.makedirs('all_drawings')

# Import the csv file with image paths and unique names
Drawing_IDs = pd.read_csv('Drawing_IDs.csv')

# make the image path column to a list
image_path_list = Drawing_IDs[["image_path"]]
image_path_list = image_path_list["image_path"].tolist()

# make the image ID column to a list
image_ID_list = Drawing_IDs[["Drawing_ID"]]
image_ID_list = image_ID_list["Drawing_ID"].tolist()

# Loop that reads the image paths and saves them into the all_drawings folder with their unique names
for i in range(len(image_ID_list)):
    drawing = cv2.imread(image_path_list[i])
    drawing_ID = "all_drawings/" + image_ID_list[i] + ".png"
    cv2.imwrite(drawing_ID, drawing)


Upload all images to google colab

Now we resize all images to 400x400px

In [ ]:
import cv2
import glob
import re
import os
#do we actually need more modules like pandas or numpy? e.g. for .format() and .sort()

 
# define path where all drawings are
path = "/content/" 
img_list = glob.glob(path + "*.png")
img_list.sort()

id = []

if not os.path.exists('resized'):
    os.makedirs('resized')

for i in range(len(img_list)):
  id.append(re.findall('/content/(Chain_\d+_Gen_\d+_Cond_\d+_Source_\d+).png', img_list[i])[0])
  src = cv2.imread(img_list[i], cv2.IMREAD_UNCHANGED)

  # set a new height and width in pixels
  new_height = 400
  new_width = 400

  # dsize
  dsize = (new_width, new_height)

  # resize image
  output = cv2.resize(src, dsize)

  cv2.imwrite('/content/resized/{}.png'.format(id[i]), output)


The resized images are blurred using cv.blur(), using the averaging function, and then converted to jpeg2000 format, the file sizes of which are put into a dataframe and saved as csv

In [ ]:
# import the necessary packages
import PIL
from PIL import Image
from pathlib import Path
import numpy as np
import pandas as pd
import glob
import os
import re
import cv2
#import cv2 as cv


# Define function
def complexity_comparison_function(img_list):
  
  # make directory to put jp2 files
  if not os.path.exists('jpeg_compressed_files'):
    os.makedirs('jpeg_compressed_files')
    
  # prepare dataframe
  columns = ['Stim_ID', 'Complexity_original', 'Complexity_convolution']
  index = np.arange(0)
  Complexity_comparison_data = pd.DataFrame(columns=columns, index = index)

  # list for id's
  id = []
  
  # convert png to jp2
  for i in range(len(img_list)):
    print(img_list) 
    id.append(re.findall('/content/resized/(Chain_\d+_Gen_\d+_Cond_\d+_Source_\d+).png', img_list[i])[0]) 
    
    #complexity of original:
    img_original = Image.open(img_list[i])
    img_original.convert("RGBA").save("jpeg_compressed_files_comparison/Original_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])
    
    # read image for blurring
    img_blur = cv.imread(img_list[i])

    # convolution
    kernel = np.ones((5,5),np.float32)/25
    dst = cv.filter2D(img_blur,-1,kernel)
    cv2.imwrite('/content/Convolution_{}.png'.format(id[i]), dst)

    # jpeg2000 convertion of blurred images
    img_convolution = Image.open("/content/Convolution_{}.png".format(id[i]))
    img_convolution.convert("RGBA").save("jpeg_compressed_files_comparison/Convolution_{}.jp2".format(id[i]), 'JPEG2000', quality_mode='dB', quality_layers=[80])

  # list of original jp2 files
  img_jp2_original = glob.glob("jpeg_compressed_files_comparison/Original_*.jp2")
  img_jp2_original.sort() #important! Else the IDs and complexity scores won't match

  # list of convolution jp2 files
  img_jp2_convolution = glob.glob("jpeg_compressed_files_comparison/Convolution_*.jp2")
  img_jp2_convolution.sort()

  # save size and ID to dataframe
  for i in range(len(img_jp2_original)): 
    size_original = (Path(img_jp2_original[i]).stat().st_size)
    size_convolution = (Path(img_jp2_convolution[i]).stat().st_size)

    Complexity_comparison_data = Complexity_comparison_data.append({
      'Stim_ID': id[i],
      'Complexity_original': size_original, 
      'Complexity_convolution': size_convolution
    }, ignore_index=True)
  return Complexity_comparison_data


# define path where all drawings are
path = "/content/resized/" 
files = glob.glob(path + "*.png")
files.sort() #important! Else the IDs and complexity scores won't match

# run function on all drawings
complexity_comparison_data = complexity_comparison_function(files)

# print complexity scores and ID's
print(complexity_comparison_data)

# prepare logfilename and save
logfilename = "complexity_comparison.csv"
complexity_comparison_data.to_csv(logfilename)
